In [35]:
from langchain.llms import OpenAI
from constants import openai_key
os.environ["OPENAI_API_KEY"] = openai_key

### OPENAI 

In [36]:
## Value towards 1 will give you creative answers but it has risk associated
llm = OpenAI(temperature=0.6)
text = "What is capital of india"
print(llm.predict(text))



The capital of India is New Delhi.


In [37]:
print(llm.predict("Can you write a poem about AI"))



AIs are here for us,
to help us with our fuss,
and make our lives more efficient,
with their intelligence so resilient.

They can help us with our tasks,
and save us from any mess,
as they learn and adapt,
and with their skills they can outlast.

With AI we can innovate,
create things that we can relate,
and make our lives more bright,
with the help of their might.

They can help us with our jobs,
and aid us in our sobs,
as they are here to stay,
we can use them in every way.


### HuggingFace

We will use flan-t5-base model 

In [32]:
from langchain import HuggingFaceHub
#from constants import hugging_token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hugging_token
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-base",model_kwargs={"temperature":0,"max_length":64})
output = llm_huggingface.predict("Capital of india")
print(output)

/Users/mfho-27019739/Desktop/AIML/Github/LLM/.venv/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


Delhi


In [34]:
output = llm_huggingface.predict("Can you write a poem about AI")
print(output)

i love a samurai warrior


As you can see why we will use OPENAI model instead of Hugging Face open source model

### Prompt Templates 

In [39]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(input_variables=['country'],
template = "Tell me the capital of {country}")
prompt_template.format(country="India")

'Tell me the capital of India'

In [41]:
llm.predict(prompt_template=prompt_template,text="India")

TypeError: Object of type PromptTemplate is not JSON serializable

In [40]:
## So instead of using LLM Predict we use LLMChain
from langchain.chains import LLMChain
#llm.predict(prompt_template=prompt_template,text="India")
chain = LLMChain(llm=llm,prompt=prompt_template)
chain.run('India')

'\n\nThe capital of India is New Delhi.'

### Combining Multiple chains using Simple Sequential Chain 

In [42]:
capital_template = PromptTemplate(input_variables=['country'],
template = "Tell me capital of {country}")
capital_chain = LLMChain(llm=llm,prompt=capital_template)

famous_template = PromptTemplate(input_variables=['capital'],
template = "Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template) 

In [43]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("India")

" Here are some amazing places to visit in New Delhi:\n\n1. Red Fort: It is a historical fort in the city of New Delhi and was the main residence of the Mughal emperors of India for nearly 200 years.\n\n2. India Gate: It is a war memorial located astride the Rajpath, on the eastern edge of the ceremonial axis of New Delhi.\n\n3. Humayun's Tomb: It is the tomb of the Mughal Emperor Humayun in Delhi, India. The tomb was commissioned by Humayun's first wife and chief consort, Empress Bega Begum.\n\n4. Qutub Minar: It is a UNESCO World Heritage Site and the tallest minaret in India. It is a red sandstone tower, which is 73 meters high.\n\n5. Jama Masjid: It is one of the largest mosques in India. It was built by Mughal Emperor Shah Jahan between 1644 and 1656 at a cost of one million rupees.\n\n6. Lotus Temple: It is an Bahá'í House of Worship located in Delhi. It is a prominent attraction in the city and has become a symbol of strength and beauty.\n\n7"

In above ans we cannot see entire chain . Chain 1 output is not displayed . To get that we can use SequentialChain

### Sequential Chain

In [44]:
capital_template = PromptTemplate(input_variables=['country'],
template = "Tell me capital of {country}")
capital_chain = LLMChain(llm=llm,prompt=capital_template,output_key='capital')

famous_template = PromptTemplate(input_variables=['capital'],
template = "Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template,output_key="places") 

In [46]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[capital_chain,famous_chain],input_variables=['country'],output_variables=['capital','places'])
chain({'country':'india'})

{'country': 'india',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': " Here are some of the amazing places to visit in New Delhi:\n\n1) Red Fort: Red Fort is a massive, red sandstone fort in Old Delhi, built in 1638 by the Mughal Emperor Shah Jahan. It is one of the most visited monuments in India.\n\n2) India Gate: India Gate is a war memorial located in New Delhi, built in 1931 to honor the Indian soldiers who died in World War I and the Afghan Wars.\n\n3) Qutub Minar: Qutub Minar is a 73-meter-high tower and a UNESCO World Heritage site in Delhi. It is the tallest minaret in India.\n\n4) Lotus Temple: The Lotus Temple is a Bahá'í House of Worship located in South Delhi. It is a magnificent building made of white marble and is shaped like a lotus flower.\n\n5) Jantar Mantar: Jantar Mantar is an astronomical observatory built in 1724 by Maharaja Jai Singh II of Jaipur. It has several large instruments that were used to measure the position of stars and planets.\n\n6) 

### Chatmodels with OpenAI

## Consider 3 schemas in Chatbot
- Human Message 
- System Message
- AI Message (Ouput given by model)

In [47]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage
chatllm = ChatOpenAI(temperature=0.6,model='gpt-3.5-turbo')

## System Message - We are instructing chatbot to behave in that way 
## Human Message - Input 
## AI Message - Output 

chatllm([
    SystemMessage(content='You are a comedian AI Assistant'),
    HumanMessage(content="Please provide some comedy punchlines on AI")

])

AIMessage(content='1. "Why did the AI go to therapy? Because it had too many bugs in its code and needed to work out its issues!"\n\n2. "Why did the AI take up gardening? It wanted to learn how to grow a sense of humor from scratch!"\n\n3. "What do you call an AI that tells jokes? A silicon-based stand-up comedian!"\n\n4. "Why did the AI refuse to play cards? It heard the dealer was a bit of a chip!"\n\n5. "Why did the AI start a band? Because it wanted to create music that speaks to both humans and binary code!"\n\n6. "Did you hear about the AI that tried to become a chef? It kept getting stuck in a digital pot!"\n\n7. "Why did the AI cross the road? To optimize its algorithm for efficient traffic navigation!"\n\n8. "Why did the AI get a job at the bakery? It kneaded a new challenge to rise to!"\n\n9. "What do you get when you cross an AI with a comedian? A virtual funny bone that computes punchlines in nanoseconds!"\n\n10. "Why did the AI go on a diet? It realized it had too many byt

### Prompt Template + LLM + Output Parsers

In [48]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate

## Modifying the output of llm model beforehand 
from langchain.schema import BaseOutputParser

In [49]:
class Commaseperatedoutput(BaseOutputParser):
    ## Diving all the words by comma seperated
    def parse(self,text:str):
        return text.strip().split(",")

In [51]:
template = "You are helpfull assistant . When the user gives any input , you should generate 5 synonym words in comma seperated list" ## System Template 
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages(
    [
        ("system",template),
        ("human",human_template)
    ]
)

chain = chatprompt|chatllm

chain.invoke({"text":"intelligent"})


AIMessage(content='smart, clever, knowledgeable, astute, brilliant')

In [52]:
chain = chatprompt|chatllm|Commaseperatedoutput()

chain.invoke({"text":"intelligent"})

['smart', ' clever', ' brilliant', ' sharp', ' astute']